In [1]:
from src.utils.spark_utils import get_spark_session

In [2]:
spark=get_spark_session()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 11:26:03 WARN Utils: Your hostname, vivekpanthi-Swift-SF514-55TA, resolves to a loopback address: 127.0.1.1; using 192.168.254.14 instead (on interface wlp0s20f3)
25/11/19 11:26:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/myvenv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/vivekpanthi/.ivy2.5.2/cache
The jars for the packages stored in: /home/vivekpanthi/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3778cd00-6592-4f0c-be20-d90f5e48bca1;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: 

In [ ]:
df=spark.read.format("delta").option("header",True).load("/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/data/bronze/bronze_customers_raw")

In [6]:
df.show(50,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
BRONZE_TABLE_CUSTOMERS="bronze_ingestion_date"

In [10]:
df.select("payment_method").distinct().show()

+----------------+
|  payment_method|
+----------------+
|       gift_card|
|             cod|
|     Credit Card|
|          paypal|
|     CREDIT_CARD|
|          PayPal|
|     credit_card|
|          PAYPAL|
|      debit_card|
|Cash on Delivery|
|            NULL|
+----------------+



In [17]:
    df=spark.sql(f"""
        SELECT 
            customer_id,
            email,
            customer_segment,
            _dq_has_errors,
            _dq_error_count,
            _bronze_ingestion_timestamp
        FROM {BRONZE_TABLE_CUSTOMERS}
        ORDER BY _bronze_ingestion_timestamp DESC
        LIMIT 5
    """).show(truncate=False)

{"ts": "2025-11-17 01:19:09.756", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o37.sql.\n: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIE

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 9 pos 9;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Sort ['_bronze_ingestion_timestamp DESC NULLS LAST], true
      +- 'Project ['customer_id, 'email, 'customer_segment, '_dq_has_errors, '_dq_error_count, '_bronze_ingestion_timestamp]
         +- 'UnresolvedRelation [bronze_ingestion_date], [], false


In [4]:
import os

bronze_base = "./data/bronze"

table_names = [
    d for d in os.listdir(bronze_base)
    if os.path.isdir(os.path.join(bronze_base, d))
]

print(table_names)


['bronze_customers_raw']
